In [3]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 40.9 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=ff4f52d47904633d9ee18e254d3cc405cdfec2564a041280b0e72f79eafacadf
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
pip install yahoofinancials

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=9285e6463ee3d1ac8351301885836b2b58a451a044b5f402af8dfdfe3ed71a1f
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [5]:
import tweepy
from textblob import TextBlob
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import datetime as datetime
plt.style.use('fivethirtyeight')
import yfinance as yf
from yahoofinancials import YahooFinancials

In [6]:
authenticate = tweepy.OAuthHandler('nGl7z1dOahjmkBy3JyMzbW97t','DxUTVVB8g943XMWxK6TdSffrPKCJ4Vkl8Z6MRkV6tN0M9hC4l6')
authenticate.set_access_token('955136760161292288-WBASBfndFoMMTPGUBszS1Yvb0agAdvk', 'a49eF9i5zfTGomSu9QPW2eBNCt1Fx1kS9M6MS3XYNhJa8')
api = tweepy.API(authenticate, wait_on_rate_limit = True)
#AAAAAAAAAAAAAAAAAAAAAAEqTgEAAAAAw32BQBEt8LwFC5ksf0OAXfp10m0%3DohzsGoZKgzdgCl9h9MsKWlWmNE8MHHkPw54w5qyEjzltIz6HqI



In [7]:
def cleanTweet(twt):
  twt = re.sub('#[A-Za-z0-9]+','',twt)
  twt = re.sub('\\n','', twt)
  twt = re.sub('https?:\/\/\S+', '', twt)
  return twt

In [8]:
def getStock(stkr):  
  ps = yf.download(stkr, 
                      start='2021-09-16', 
                      end='2021-09-27', 
                      progress=False,
  )


  ps = ps[['Adj Close']]
  ps['Shift'] = ps['Adj Close'].shift(1)
  ps['Diff'] = ps['Adj Close'] - ps['Shift']
  ps['% Change'] = ps['Diff']/ps['Adj Close']
  big = ps['% Change'] * 100
  #lol.to_frame
  #lol.reindex(index=lol.index[::-1])
  #big.iloc[1:]
  #print(big)
  big = big.tolist()
  big.pop(0)
  return big


In [28]:
def getNumSents(stock, dateOf):
  search_term = '#'+stock+' -filter:retweets'
  dateprev = datereducer(dateOf,-1)
  tweets = tweepy.Cursor(api.search, q=search_term, lang = 'en', since = dateprev , until = dateOf , tweet_mode = 'extended').items(2000)
  all_tweets = [[tweet.full_text, tweet.created_at] for tweet in tweets]
  df = pd.DataFrame(all_tweets, columns = ['Tweets', 'Time'])



  df['Cleaned'] = df['Tweets'].apply(cleanTweet)
  df['Subjectivity'] = df['Cleaned'].apply(getSubjectivity)
  df['Polarity'] = df['Cleaned'].apply(getPolarity)
  df['Sentiment'] = df['Polarity'].apply(getSentiment)
  
  return df


In [10]:
#needs to be updated so that it considers month and year
#may be able to use datetime methods
def datereducer(string, num):
  day = string[8:10]
  eday = string[0:8]
  day = int(day)
  newday = day + num
  newday = str(newday)
  return (eday+newday)




In [16]:
def getSentsDf(stock):
  num = pd.DataFrame()
  i = 0
  date = '2021-09-28'
  List1 = []
  List2 = []
  List3 = []
  while(i < 8):
    weekno = datetime.datetime.strptime(date, '%Y-%m-%d').date().weekday()
    if(weekno < 5):
      
      df = getNumSents(stock,date)
      t = df['Sentiment'].value_counts(normalize = True)[0]
      t2 = df['Sentiment'].value_counts(normalize = True)[1]

      
    
    #if(weekno == 5):
      #List3.append(datereducer(date,2))
      
    #elif(weekno == 6):
      #List3.append(datereducer(date,1))
      List1.append(t)
      List2.append(t2)
      List3.append(weekno)

    date = datereducer(date, -1)
    i += 1
  num['Positive'] = List1
  num['Negative'] = List2
  num['Date'] = List3
  num['Price'] = getStock(stock)
  return num
  



In [12]:
def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity

def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity

In [13]:
def getSentiment(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'


In [35]:
aapl = getSentsDf('AAPL')
aapl['Price'] = getStock('AAPL')
#print(aapl)
ge = getSentsDf('GE')
ge['Price'] = getStock('GE')
#print('GE')
frame = ge.append([aapl])
#print(frame)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [36]:
#Use to gather dataset of sotck sentiment
def accumulate(doink):
    pp = pd.DataFrame()
    for index, row in doink.iterrows():
      pp = pp.append(getSentsDf(row['Symbol']))
    return pp


In [37]:
x = frame.drop('Price', axis = 1)
y = frame[["Price"]]
y.head()
x.head()
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(x,y)

In [38]:
def predict(stikr, date):
  gg = getNumSents(stikr, date)
  t = gg['Sentiment'].value_counts(normalize = True)[0]
  t2 = gg['Sentiment'].value_counts(normalize = True)[1]
  t3 = weekno = datetime.datetime.strptime(date, '%Y-%m-%d').date().weekday()
  new_data = [[t],[t2],[t3]]
  new_data = pd.DataFrame(new_data).T
  return model.predict(new_data)

In [39]:

predict('AAPL', '2021-09-28')

array([[0.63050511]])

In [40]:
ps = yf.download('AAPL', 
                      start='2021-09-24', 
                      end='2021-09-29', 
                      progress=False,
  )


ps = ps[['Adj Close']]
ps['Shift'] = ps['Adj Close'].shift(1)
ps['Diff'] = ps['Adj Close'] - ps['Shift']
ps['% Change'] = ps['Diff']/ps['Adj Close']
print(ps)

             Adj Close       Shift      Diff  % Change
Date                                                  
2021-09-24  146.919998         NaN       NaN       NaN
2021-09-27  145.369995  146.919998 -1.550003 -0.010662
2021-09-28  141.910004  145.369995 -3.459991 -0.024382


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [23]:
url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv'
sp=pd.read_csv(url)

In [31]:
lol = sp.loc[sp['Symbol'] == 'AAPL']
lol = lol.append(sp.loc[sp['Symbol'] == 'AMZN'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'ZBRA'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'ZION'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'PYPL'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'ORCL'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'ATVI'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'ADBE'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'AMD'])
#lol = lol.append(sp.loc[sp['Symbol'] == 'GOOGL'])


In [ ]:
bbb = sp.sample(frac=0.25,random_state=1)

COP
ORCL
TXN


['', ' a hello ', ' Hi']

In [ ]:
sp

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [ ]:
plt.figure(figsize = (8,6))
for i in range(0,df.shape[0]):
    plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color = 'Purple')
plt.title('Sentiment Analysis Scatter Plot')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

NameError: ignored

<Figure size 576x432 with 0 Axes>

In [ ]:
df['Sentiment'].value_counts().plot(kind = 'bar')
plt.title('Sentiment Analysis Bar Plot')
plt.xlabel('Sentiment')
plt.ylabel('Number of Tweets')
plt.show()

NameError: ignored